In [1]:
mutable struct AlgebraicArray
    f::Function
    n::Int64
    calls::Vector{Pair{Function, Tuple}}
    function AlgebraicArray(f::Function, n::Int64)
        new(f, n, [])
    end
end

In [2]:
function add_algebra(aa::Symbol, args::Any)
    aa = eval(aa)
    if length(args) > 1
        farguments = Tuple(eval(args[length(args)]))
        push!(aa.calls, eval(args[1]) => farguments)
    else
        push!(aa.calls, eval(args[1]) => [])
    end
    
end

add_algebra (generic function with 1 method)

In [3]:
function generate(aa::AlgebraicArray)
    [aa.f(n) for n in 1:aa.n]
end

generate (generic function with 1 method)

In [103]:
function generate(aa::AlgebraicArray, mask::BitArray)
    if length(mask) != aa.n
        throw(DimensionMismatch("Mask must be the same size as AlgebraicArray!"))
    end
    vals = Array(1:aa.n)
    filter!(vals, mask)
    [aa.f(n) for n in vals]
end

generate (generic function with 4 methods)

In [104]:
function generate(aa::AlgebraicArray, range::UnitRange)
    if range[2] > aa.n
        throw(BoundsError(string("Invalid algebraic index, ", string(range[2], 
                        " on algebraic expression of length ", string(aa.n)))))
    end
    [aa.f(n) for n in range]
end

generate (generic function with 4 methods)

In [105]:
function generate(aa::AlgebraicArray, index::Integer)
    if index > aa.n
        throw(BoundsError(string("Invalid algebraic index, ", string(range[2], 
                        " on algebraic expression of length ", string(aa.n)))))
    end
    aa.f(index)[1]
end

generate (generic function with 4 methods)

In [106]:
function compute(aa::AlgebraicArray, r::Any) # <- range, bitarray
    gen = generate(aa, r)
    for call in aa.calls
        gen = [call[1](val, call[2]...) for val in gen]
    end
    return(gen)
end

compute (generic function with 3 methods)

In [107]:
function compute(aa::AlgebraicArray, r::Integer) # <- Index
    gen = generate(aa, r)
    for call in aa.calls
        gen = [call[1](gen, call[2]...)
    end
    return(gen)
end

LoadError: syntax: unexpected "end"

In [108]:
function compute(aa::AlgebraicArray)
    gen = generate(aa)
    for call in aa.calls
        gen = [call[1](val, call[2]...) for val in gen]
    end
    return(gen)
end

compute (generic function with 3 methods)

In [109]:
macro algebraic!(exp::Expr)
    args = copy(exp.args)
    aa = exp.args[2]
    deleteat!(args, 2)
    add_algebra(aa, args)
end

@algebraic! (macro with 1 method)

In [110]:
function defineval(n)
    n + 1
end

defineval (generic function with 1 method)

In [111]:
tuple(5, 10, 15)

(5, 10, 15)

In [112]:
z = AlgebraicArray(defineval, 20)

AlgebraicArray(defineval, 20, Pair{Function, Tuple}[])

In [113]:
generate(z)

20-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21

In [114]:
@algebraic! z * 2

1-element Vector{Pair{Function, Tuple}}:
 (*) => (2,)

In [115]:
compute(z)

20-element Vector{Int64}:
  4
  6
  8
 10
 12
 14
 16
 18
 20
 22
 24
 26
 28
 30
 32
 34
 36
 38
 40
 42

In [116]:
import Base: getindex

In [117]:
getindex(aa::AlgebraicArray, i::Any) = compute(aa, i)

getindex (generic function with 220 methods)

In [118]:
z[5][1]

12

In [119]:
z[1:10]

10-element Vector{Int64}:
  4
  6
  8
 10
 12
 14
 16
 18
 20
 22

In [120]:
z

AlgebraicArray(defineval, 20, Pair{Function, Tuple}[(*) => (2,)])

In [121]:
function compute(f::Function, aa::AlgebraicArray) # compute(aa)
    gen = generate(aa)
    for call in aa.calls
        gen = [call[1](gen, call[2]...)]
    end
    return(f(gen))
end

compute (generic function with 3 methods)

In [122]:
mask = compute(z) do aa
    [if val > 12 true else false end for val in aa[1]]
end

20-element Vector{Bool}:
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [123]:
compute(z, BitArray(mask))

LoadError: MethodError: objects of type Vector{Int64} are not callable
Use square brackets [] for indexing an Array.

In [124]:
varinfo()

| name           |      size | summary                                      |
|:-------------- | ---------:|:-------------------------------------------- |
| @algebraic!    |   0 bytes | @algebraic! (macro with 1 method)            |
| AlgebraicArray | 160 bytes | DataType                                     |
| Base           |           | Module                                       |
| Core           |           | Module                                       |
| Main           |           | Module                                       |
| add_algebra    |   0 bytes | add_algebra (generic function with 1 method) |
| compute        |   0 bytes | compute (generic function with 3 methods)    |
| defineval      |   0 bytes | defineval (generic function with 1 method)   |
| generate       |   0 bytes | generate (generic function with 4 methods)   |
| mask           |  60 bytes | 20-element Vector{Bool}                      |
| z              | 104 bytes | AlgebraicArray                               |
